## 7.19 结构与类

如今，编程教科书推荐使用面向对象编程，使软件开发更加清晰和模块化。
对象是结构和类的实例。
面向对象的编程风格对程序性能，既有正面又有负面的影响。积极影响是：

- 如果一组变量是相同结构或类的成员，则它们一起使用，也存储在一起。这使数据缓存更有效。
- 类成员变量不需要作为参数传递给类成员函数，避免了参数传输的开销。

面向对象编程的负面影响是：

- 非静态成员函数有一个'`this`'指针，它作为隐式参数传递给函数。所有非静态成员函数都会产生'`this`'参数传输的开销。
- '`this`'指针占用一个寄存器。寄存器是32位系统中的稀缺资源。
- 虚拟成员函数效率较低（请参阅第55页）。

**关于面向对象编程的正面影响，还是负面影响占主导地位，没有一般性的定论。**
至少，可以说使用类和成员函数，代价并不昂贵。
你可以使用面向对象的编程风格：
- 如果它对程序的逻辑结构和清晰度有好处
- 只要在程序的最关键部分避免过多的函数调用，

单纯结构的使用（没有成员函数）对性能没有负面影响。

## 7.20 类数据成员（实例变量）

无论在什么时间创建类或结构的实例，其数据成员都按照它们被声明的顺序连续存储。
将数据组织到类或结构中，并不会导致性能损失。
访问类或结构对象的数据成员并不会比访问简单变量话费更多的时间。

大多数编译器会将数据成员与地址对齐，以便优化访问，如下表所示。

Type | size, bytes  | alignment, bytes
 ---  | ---  | ---  |
bool  |  1   |  1
char, signed or unsigned  | 1  | 1
short int, signed or unsigned  | 2  | 2
int, signed or unsigned  | 4  | 4
64-bit integer, signed or unsigned  | 8  | 8
pointer or reference, 32-bit mode  | 4  | 4
pointer or reference, 64-bit mode  | 8  | 8
float  | 4  | 4
double  | 8 |  8
long double  | 8, 10, 12 or 16 |  8 or 16

Table 7.2. 数据成员的对齐

结构或类中如果混有不同大小的数据成员，数据对齐可能会导致未使用字节的空洞。例如：
```cpp
// Example 7.39a
struct S1 {
   short int a; // 2个字节. 第一个在偏移位置0, 第二个在位置1
   // 6个未使用的字节
   double b; // 8个字节. 第一个在偏移位置8, 最后一个在位置15
   int d; // 4个字节. 第一个在偏移位置16, 最后一个在位置19
   // 4个未使用字节
};
S1 ArrayOfStructures[100];
```
此处数据对齐导致结构缩小了8个字节，数组缩小了800个字节。

通过重新安排数据成员的编码顺序，通常可以使结构和类对象更小。
如果类有至少一个虚拟函数，则在第一个数据成员之前，或最后一个成员之后，有一个指向虚拟表的指针。
该指针在32位系统中为4个字节，在64位系统中为8个字节。
如果你对结构或其每个成员的大小有疑问，可以使用`sizeof`运算符进行测试。
`sizeof`运算符返回的值包括对象末尾的任何未使用的字节的大小。

- **如果数据成员相对于结构或类的开头的偏移量小于128，则生成的访问数据成员的代码更为紧凑，因为偏移量可以表示为8位有符号数。**
- 如果相对于结构或类的开头的偏移量大于等于128字节，则偏移量必须表示为32位数（指令集在介于8位和32位偏移之间，没有任何区别）。 

例如：
```cpp
// Example 7.40
class S2 {
public:
   int a[100]; // 400字节. 首字节偏移量0, 尾字节偏移量399
   int b; // 4字节. 首字节偏移量400, 尾字节偏移量403
   int ReadB() {return b;}
};
```

这里`b`的偏移量为400。通过指针或成员函数（如`ReadB`）访问`b`的任何代码都需要将偏移量编码为32位数。
如果交换`a`和`b`，则可以使用编码为8位有符号数的偏移量访问这两个变量，或者根本不用偏移。
这使代码更紧凑，从而更有效地使用代码缓存。
因此，对于大型数组和其他大型对象，建议在结构或类声明中排在最后。并且最常用的数据成员放在前面。
如果不能把所有数据成员包含前128个字节内，则将最常用的成员放在前128个字节中。

## 7.21 类成员函数(方法)

每次声明或创建类的新对象时，它都将生成数据成员的新实例。
但是每个成员函数只有一个实例。函数代码不用复制多份，因为相同的代码可以应用于类的所有实例。

下面两种方式速度一样快：
- 调用结构的成员函数
- 调用简单函数，以指针（或引用）为参数，该指针向此结构

例如:

```cpp
// Example 7.41
class S3 {
public:
   int a;
   int b;
   int Sum1() {return a + b;}
};
int Sum2(S3 * p) {return p->a + p->b;}
int Sum3(S3 & r) {return r.a + r.b;}
```
`Sum1`，`Sum2`和`Sum3`这三个函数完成相同的工作，效率也相同。
如果查看编译器生成的代码，您会注意到一些编译器将为这三个函数生成完全相同的代码。
`Sum1`有一个隐含的'`this`'指针，它与`Sum2`和`Sum3`中的`p`和`r`做同样的事情。
无论你是想让函数成为类的成员，还是给它一个指针或引用，指该向类或结构，都只是编程风格的问题。
有些编译器通过在寄存器而不是堆栈中传输'`this`'，使得`Sum1`在32位Windows中的效率略高于`Sum2`和`Sum3`。

## 7.22 虚拟成员函数

虚函数用于实现多态类。
多态类的每个实例都有一个指针，该指针指向一个指针表，表中是不同版本的虚拟函数。
这个所谓的虚表用于在运行时定位到正确版本的虚拟函数。
多态性是面向对象程序比非面向对象程序效率低的主要原因之一。
如果可以避免使用虚函数，那么你可以获得面向对象编程的大部分优势，而无需支付性能成本。

- 如果函数调用语句总是调用相同版本的虚函数，调用虚拟成员函数所花费的时间比调用非虚拟成员函数所花费的时间，只多几个时钟周期。
- 如果虚函数版本发生变化，那么可能会得到10到20个时钟周期的错误预测惩罚。

虚函数调用的预测正确和错误的规则与`switch`语句相同，如第44页所述。

当在已知确定类型的对象上调用虚函数时，可以绕过调度机制。但是你不能总是依赖编译器能绕过该调度机制，即使有时候看似很明显可以绕过，它也未必能成功。
详见第75页。

只有在编译时无法知道调用哪个版本的多态成员函数时，才需要运行时多态性。
如果需要在程序的关键部分使用虚函数，那么你可以考虑下面的方法：
- 是否可以在没有多态实现该所需功能
- 或使用编译时多态实现该功能

**有时可以使用模板而不是虚函数来获得所需的多态效果。**
模板参数应该是个类，该类包含具有多个版本的函数。
此方法更快，因为模板参数始终在编译时而不是在运行时解析。
第59页上的示例7.47是执行此操作的例子。
不幸的是，语法非常糟糕，可能不值得付出努力学这样的东西。

## 7.23 运行时类型识别(RTTI)

运行时类型标识会向所有类对象添加额外信息，其效率不高。
如果编译器有RTTI选项，把它关闭，寻找其它替代方法。

## 7.24 继承

编译器对于派生类的对象的实现方式，与包含父类和子类成员的简单类的对象相同。
父类和子类的成员访问同样的快。通常，可以假定使用继承几乎没有任何性能损失。

由于以下原因，代码缓存性能可能会略有下降：

- 父类数据成员的大小将添加到子类成员的偏移量中。
访问总偏移量大于127字节的数据成员的代码，会稍微变得不那么紧凑。详见第54页。

- 父和子的成员函数通常存储在不同的模块中。
这可能会导致大量跳转，代码缓存效率降低。
解决这个问题的方法是，确保彼此相邻调用的函数也存储在彼此附近。
详情参见第90页。

从同一代中的多个父类的多重继承，会导致成员指针和虚函数的复杂化。通过指向多个基类中一个基类的指针，来访问派生类的对象时，会变得很复杂。
你可以在派生类中创建对象，来避免多重继承：

```cpp
// Example 7.42a. 多重继承
class B1; class B2;
class D : public B1, public B2 {
public:
   int c;
};
```
替换为:
```cpp
// Example 7.42b. 多重继承替代方案
class B1; class B2;
class D : public B1 {
public:
   B2 b2;
   int c;
};
```

## 7.25 构造函数和析构函数

构造函数在内部实现为成员函数，该函数返回该对象的引用。
新对象的内存分配不一定由构造函数自身完成。
因此，构造函数与任何其他成员函数效率没有区别。
此结论适用于默认构造函数，拷贝构造函数和任何其它构造函数。

一个类可以不需要构造函数。
- 如果对象不需要初始化，可以没有默认构造函数。
- 如果只需复制所有数据成员就可以复制对象，则不需要拷贝构造函数。

简单的构造函数可以被内联，以提高性能。

当通过赋值，函数参数或函数返回值来复制对象时，拷贝构造函数都可能被调用。
如果拷贝构造函数涉及内存或其他资源的分配，可能要花费一定时间。
有多种方法可以避免这种内存复制浪费，例如：

- 使用指向对象的引用或指针，而不是复制对象本身。
- 使用“移动构造函数”来转移内存块的所有权。这需要一个支持C++ 0x的编译器。
- 创建一个类的成员函数，或者友元函数，或者运算符，将内存块的所有权从一个对象转移到另一个对象。
失去内存块所有权的对象应将其指针设置为`NULL`。
当然应该有一个析构函数来销毁对象拥有的任何内存块。

析构函数与成员函数效率相同。如果没有必要，不要写析构函数。
虚拟析构函数与虚拟成员函数效率相同。详见第55页。